In [ ]:
import pandas as pd
import numpy as np
from advanced_rag import AdvancedRAG, EmbeddingModelType, ChainType
from basic_rag import answer_query as basic_answer_query
# from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import time
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict

In [ ]:
def embedding_model_evaluate_answer(pred: str, gold: str, semantic_model) -> float:
    emb_pred = semantic_model.encode(pred, convert_to_tensor=True)
    emb_gold = semantic_model.encode(gold, convert_to_tensor=True)
    sim = util.cos_sim(emb_pred, emb_gold).item()
    return sim

def llm_evaluate_answer(pred: str, gold: str, llm, llm_prompt_list: list) -> float:
    sim_score = 0
    for prompt in llm_prompt_list:
        s = llm.get_direct_llm_answer(prompt.replace("{gold_answer}", gold).replace("{predicted_answer}", pred))
        try:
            s = float(s)
        except:
            return np.nan
        try:
            sim_score += s
        except:
            return np.nan
    sim_score /= len(llm_prompt_list)
    return sim_score

def compute_accuracy(results_df, group_col):
    metrics = results_df.groupby(group_col)["Correct"].mean().reset_index()
    metrics.columns = [group_col, "Accuracy"]
    return metrics

def evaluate_rag(rag_query_fn, questions_df, semantic_model, self_llm, llm_prompts_list, direct_llm=False):
    results = []
    for _, row in questions_df.iterrows():
        q, gold, category, no = row["Question"], row["Answer"], row["Category"], row["no"]
        if direct_llm:
            q = "Please answer based on the book '1984' written by George Orwell. " + q
        pred = rag_query_fn(q)
        embeddings_models_score = embedding_model_evaluate_answer(pred, gold, semantic_model)
        llm_score = llm_evaluate_answer(pred, gold, self_llm, llm_prompts_list)
        results.append({
            "question_no": no,
            "Predicted Answer": pred,
            "Category": category,
            "embeddings_models_score": embeddings_models_score,
            "llm_score": llm_score
        })

    results_df = pd.DataFrame(results)
    return results_df

def compute_metrics(results_df):
    category_acc = compute_accuracy(results_df, "Category")

    print("\n=== Overall Accuracy ===")
    print(results_df["Correct"].mean())

    print("\n=== Accuracy by Category ===")
    print(category_acc)

General constants

In [ ]:
self_llm = AdvancedRAG()
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
questions_df = pd.read_json("evaluation/1984_test_questions.json")

llm_prompts_list = [
    "Please score the level of correctness of the predicted answer based on the gold answer. You should give a high score to an answer that contains at least the elements contained in the gold answer. If a few additionnary elements are present in the predicted answer, you can still give a high score for the answer. If elements of the predicted answer are contradicting the gold answer, the final score should be low. 1 is a perfect score (all elements aôf the gold answer a in the predicted answer). O is a very low score (None of the elements of the gold answer are in the predicted answer).\n Gold answer : {gold_answer}. Predicted answer : {predicted_answer}. Please only give your score with no comments, explanations, side notes or post scriptum. Please do not write anything else than th score. (for example: '0.7').",

    "Please score the level of correctness of the predicted answer based on the gold answer. You should give a high score to an answer that contains at least the elements contained in the gold answer. Do not penalize additionnary elements that are present in the answer but not in the answer. If elements of the predicted answer are contradicting the gold answer, the final score should be low. 1 is a perfect score (all elements of the gold answer a in the predicted answer). O is a very low score (None of the elements of the gold answer are in the predicted answer).\n Gold answer : {gold_answer}. Predicted answer : {predicted_answer}. Please only give your score with no comments, explanations, side notes or post scriptum. Please do not write anything else than th score. (for example: '0.7')."
]

Evaluate basic RAG

In [ ]:
results_df = evaluate_rag(rag_query_fn=basic_answer_query, questions_df=questions_df.copy(), semantic_model=semantic_model, self_llm=self_llm, llm_prompts_list=llm_prompts_list)

results_df.to_json(f"evaluation/basic_rag_eval_results.json", orient="records", indent=2)

Evaluate direct LLM

In [ ]:
results_df = evaluate_rag(rag_query_fn=AdvancedRAG().get_direct_llm_answer, questions_df=questions_df.copy(), semantic_model=semantic_model, self_llm=self_llm, llm_prompts_list=llm_prompts_list, direct_llm=True)

results_df.to_json(f"evaluation/directLLM_eval_results.json", orient="records", indent=2)

Evaluate Advanced RAG with various parameters

In [ ]:
for temperature in [0.1, 0.2, 0.8, 1.2]:
    for embedding_model in [EmbeddingModelType.HuggingFace, EmbeddingModelType.Ollama]:
        for chain_type in [ChainType.STUFF, ChainType.REFINE, ChainType.MAP_REDUCE]:
            for compression in [True, False]:
                for nb_chunks in [1, 3, 5]:
                    for prompt_type in []:
                        print("---------------------------------------")
                        print(time.now())
                        print(temperature, embedding_model.value, chain_type.value, compression, nb_chunks)
                        Advanced_rag = AdvancedRAG(compression=compression,
                                                   embedding_model_type=embedding_model,
                                                   chain_type=chain_type,
                                                   nb_chunks=nb_chunks,
                                                   llm_temperature=temperature)
                        try:
                            results_df = evaluate_rag(rag_query_fn=Advanced_rag.answer_query,
                                                      questions_df=questions_df.copy(),
                                                      semantic_model=semantic_model,
                                                      self_llm=self_llm,
                                                      llm_prompts_list=llm_prompts_list)

                            results_df["compression"] = compression
                            results_df["embedding_model"] = embedding_model.value
                            results_df["chain_type"] = chain_type.value
                            results_df["nb_chunks"] = nb_chunks
                            results_df["temperature"] = temperature

                            results_df.to_json(f"evaluation/advanced_rag_compression_{str(compression)}_embModel_{embedding_model.value}_chainType_{chain_type.value}_nbChunks_{str(nb_chunks)}_LLMtemp_{str(temperature)}_eval_results.json", orient="records", indent=2)
                        except Exception as e:
                            print(e)

In [ ]:
for file in os.listdir("evaluation/"):
    if "test_questions" in file or file.endswith(".png"):
        continue

    # Load the JSON data
    with open("evaluation/directLLM_eval_results.json", "r") as f:
        data = json.load(f)

    # Group scores by category
    categories = defaultdict(lambda: {"embedding": [], "llm": []})

    for item in data:
        cat = item["Category"]
        categories[cat]["embedding"].append(item["embeddings_models_score"])
        categories[cat]["llm"].append(item["llm_score"])

    # Prepare plotting data
    cat_names = list(categories.keys())

    embedding_scores = [vals["embedding"] for vals in categories.values()]
    llm_scores = [vals["llm"] for vals in categories.values()]

    # Add mean as the last value per category
    embedding_with_means = [scores + [np.mean(scores)] for scores in embedding_scores]
    llm_with_means = [scores + [np.mean(scores)] for scores in llm_scores]

    # Plot setup
    fig, axes = plt.subplots(2, 1, figsize=(10, 10), sharex=True)

    # Define colors
    color_individual = "C0"
    color_mean = "C3"

    for ax, scores, title in zip(
        axes, [embedding_with_means, llm_with_means],
        ["Embedding Scoring Method", "LLM Comparison Scoring Method"]
    ):
        bar_width = 0.1
        x = np.arange(len(cat_names))

        for i, cat_scores in enumerate(scores):
            offsets = np.linspace(-bar_width*(len(cat_scores)-1)/2,
                                  bar_width*(len(cat_scores)-1)/2, len(cat_scores))
            for j, val in enumerate(cat_scores):
                color = color_mean if j == len(cat_scores)-1 else color_individual
                bar = ax.bar(x[i] + offsets[j], val, width=bar_width, color=color)
                # Add label on top of bar
                ax.text(
                    x[i] + offsets[j],
                    val + 0.02,
                    f"{val:.2f}",
                    ha='center',
                    va='bottom',
                    fontsize=8,
                    rotation=90
                )

        ax.set_title(title, fontsize=14)
        ax.set_ylabel("Score")
        ax.set_ylim(0, 1.1)
        ax.grid(axis="y", linestyle="--", alpha=0.6)

    # Add legend using proxy artists with correct colors
    ind_bar = plt.Rectangle((0, 0), 1, 1, color=color_individual)
    mean_bar = plt.Rectangle((0, 0), 1, 1, color=color_mean)
    axes[0].legend([ind_bar, mean_bar], ["Individual Scores", "Mean"], loc="upper right")

    # Common X labels
    axes[-1].set_xticks(np.arange(len(cat_names)))
    axes[-1].set_xticklabels(cat_names, rotation=30, ha="right")

    plt.suptitle(f"RAG evaluation results for {file.split('_eval_results.json')[0]}")

    plt.tight_layout()
    plt.savefig(f"evaluation/{file.split('_eval_results.json')[0]}.png")
